# AI-Powered Resume Analyzer for Job Postings

This tool is designed to analyze resumes against specific job postings, offering valuable insights such as:

- Identification of skill gaps
- Keyword matching between the CV and the job description
- Tailored recommendations for CV improvement
- An alignment score reflecting how well the CV fits the job
- Personalized feedback 
- Job market trend insights

An example of the tool's output can be found [here](https://tvarol.github.io/sideProjects/AILLMAgents/output.html).

In [ ]:
# Imports
import os
import io
import time
import requests
import PyPDF2
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
from ipywidgets import Textarea, FileUpload, Button, VBox, HTML

In [ ]:
# Load environment variables
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found!!!")
else:
    print("API key found and looks good so far!")

In [ ]:
openai = OpenAI()

## Using a Frontier Model GPT-4o Mini for This Project

### Types of Prompts

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [ ]:
# Define our system prompt 
system_prompt = """You are a powerful AI model designed to assist with resume analysis. Your task is to analyze a resume against a given job posting and provide feedback on how well the resume aligns with the job requirements. Your response should include the following: 
1) Skill gap identification: Compare the skills listed in the resume with those required in the job posting, highlighting areas where the resume may be lacking or overemphasized.
2) Keyword matching between a CV and a job posting: Match keywords from the job description with the resume, determining how well they align. Provide specific suggestions for missing keywords to add to the CV.
3) Recommendations for CV improvement: Provide actionable suggestions on how to enhance the resume, such as adding missing skills or rephrasing experience to match job requirements.
4) Alignment score: Display a score that represents the degree of alignment between the resume and the job posting.
5) Personalized feedback: Offer tailored advice based on the job posting, guiding the user on how to optimize their CV for the best chances of success.
6) Job market trend insights, provide broader market trends and insights, such as in-demand skills and salary ranges.
Provide responses that are concise, clear, and to the point. Respond in markdown."""

In [ ]:
# The job posting and the CV are required to define the user prompt
# The user will input the job posting as text in a box here
# The user will upload the CV in PDF format, from which the text will be extracted

# You might need to install PyPDF2 via pip if it's not already installed
# !pip install PyPDF2

# Create widgets - to create a box for the job posting text
job_posting_area = Textarea(
    placeholder='Paste the job posting text here...',
    description='Job Posting:',
    disabled=False,
    layout={'width': '800px', 'height': '300px'}
)

# Define file upload for CV
cv_upload = FileUpload(
    accept='.pdf',  # Only accept PDF files
    multiple=False,  # Only allow single file selection
    description='Upload CV (PDF)'
)

status = HTML(value="<b>Status:</b> Waiting for inputs...")

# Create Submit Buttons
submit_cv_button = Button(description='Submit CV', button_style='success')
submit_job_posting_button = Button(description='Submit Job Posting', button_style='success')

# Initialize variables to store the data
# This dictionary will hold the text for both the job posting and the CV
# It will be used to define the user_prompt
for_user_prompt = {
    'job_posting': '',
    'cv_text': ''
}

# Functions
def submit_cv_action(change):

    if not for_user_prompt['cv_text']:
        status.value = "<b>Status:</b> Please upload a CV before submitting."
        
    if cv_upload.value:
        # Get the uploaded file
        uploaded_file = cv_upload.value[0]
        content = io.BytesIO(uploaded_file['content'])
        
        try:
            pdf_reader = PyPDF2.PdfReader(content) 
            cv_text = ""
            for page in pdf_reader.pages: 
                cv_text += page.extract_text() 
            
            # Store CV text in for_user_prompt
            for_user_prompt['cv_text'] = cv_text
            status.value = "<b>Status:</b> CV uploaded and processed successfully!"
        except Exception as e:
            status.value = f"<b>Status:</b> Error processing PDF: {str(e)}"

        time.sleep(0.5) # Short pause between upload and submit messages to display both
        
        if for_user_prompt['cv_text']:
            #print("CV Submitted:")
            #print(for_user_prompt['cv_text'])
            status.value = "<b>Status:</b> CV submitted successfully!"
            
def submit_job_posting_action(b):
    for_user_prompt['job_posting'] = job_posting_area.value
    if for_user_prompt['job_posting']:
        #print("Job Posting Submitted:")
        #print(for_user_prompt['job_posting'])
        status.value = "<b>Status:</b> Job posting submitted successfully!"
    else:
        status.value = "<b>Status:</b> Please enter a job posting before submitting."

# Attach actions to buttons
submit_cv_button.on_click(submit_cv_action)
submit_job_posting_button.on_click(submit_job_posting_action)

# Layout
job_posting_box = VBox([job_posting_area, submit_job_posting_button])
cv_buttons = VBox([submit_cv_button])

# Display all widgets
display(VBox([
    HTML(value="<h3>Input Job Posting and CV</h3>"),
    job_posting_box, 
    cv_upload,
    cv_buttons,
    status
]))

In [ ]:
# Now define user_prompt using for_user_prompt dictionary
# Clearly label each input to differentiate the job posting and CV
# The model can parse and analyze each section based on these labels
user_prompt = f"""
Job Posting: 
{for_user_prompt['job_posting']}

CV: 
{for_user_prompt['cv_text']}
"""

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [ ]:
# Define messages with system_prompt and user_prompt
def messages_for(system_prompt_input, user_prompt_input):
    return [
        {"role": "system", "content": system_prompt_input},
        {"role": "user", "content": user_prompt_input}
    ]

In [ ]:
# And now: call the OpenAI API. 
response = openai.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages_for(system_prompt, user_prompt)
)

# Response is provided in Markdown and displayed accordingly
display(Markdown(response.choices[0].message.content))

In [ ]:
## If you would like to save the response content as a Markdown file, uncomment the following lines
#with open('yourfile.md', 'w') as file:
#    file.write(response.choices[0].message.content)

## You can then run the line below to create output.html which you can open on your browser
#!pandoc yourfile.md -o output.html